In [ ]:
# === CELDA DE CONEXIÓN (ejecutar una sola vez) ===
import comtypes.client

# Conectar a instancia activa de SAP2000
try:
    SapObject = comtypes.client.GetActiveObject("CSI.SAP2000.API.SapObject")
    SapModel = SapObject.SapModel
    print("✅ Conectado a SAP2000")
    print(f"📁 Archivo: {SapModel.GetModelFilename()}")
except Exception as e:
    print(f"❌ Error de conexión: {e}")
    print("   Asegúrate de que SAP2000 esté abierto.")

In [ ]:
def coordinate_range(SapModel, xmin, xmax, ymin, ymax, zmin, zmax,
                     deselect=False, csys="Global", include_intersections=False,
                     point=True, line=True, area=True, solid=True, link=True):
    """
    Wrapper para SapModel.SelectObj.CoordinateRange con manejo de retornos de comtypes.
    Retorna (ok: bool, ret_raw)
    """
    try:
        ret = SapModel.SelectObj.CoordinateRange(
            float(xmin), float(xmax),
            float(ymin), float(ymax),
            float(zmin), float(zmax),
            bool(deselect),
            str(csys),
            bool(include_intersections),
            bool(point), bool(line), bool(area), bool(solid), bool(link)
        )
    except Exception as e:
        return False, e

    # ret puede ser int o tuple/list cuyo último elemento es RetCode
    if isinstance(ret, (list, tuple)):
        rc = int(ret[-1])
        return (rc == 0), ret
    else:
        return (int(ret) == 0), ret
    
# Ejemplo de uso:
h_col = 250
b_col = 200
z_col = 0  # o el valor que quieras
ok, ret = coordinate_range(
    SapModel,
    -b_col/2, b_col/2,    # Xmin, Xmax
    h_col/2, h_col/2,     # Ymin, Ymax (rango 0 altura fija)
    z_col, z_col,         # Zmin, Zmax (plano Z)
    deselect=False,
    csys="Global",
    include_intersections=True,
    point=True, line=False, area=False, solid=False, link=False
)
if not ok:
    print("CoordinateRange falló:", ret)

In [48]:
def divide_area_by_edge_points(SapModel, area_name, n1=2, n2=2, maxsize1=0, maxsize2=0):
    """
    Divide `area_name` usando puntos seleccionados en los bordes (MeshType=3).
    Requiere que los puntos a usar ya estén seleccionados en el modelo.
    Retorna (ok: bool, area_names: list or None, raw_ret).
    """
    try:
        # Firma: Divide(Name, MeshType, NumberAreas, AreaName(), n1, n2, MaxSize1, MaxSize2,
        #               PointOnEdgeFromGrid, PointOnEdgeFromLine, PointOnEdgeFromPoint, ...)
        ret = SapModel.EditArea.Divide(
            str(area_name),
            3,          # MeshType = 3 -> puntos en bordes
            0,          # placeholder NumberAreas (ByRef)
            [],         # placeholder AreaName() (ByRef)
            int(n1),
            int(n2),
            float(maxsize1),
            float(maxsize2),
            False,      # PointOnEdgeFromGrid
            False,      # PointOnEdgeFromLine
            True        # PointOnEdgeFromPoint -> usar puntos seleccionados en los bordes
        )
    except Exception as e:
        return False, None, e

    # ret may be int or [NumberAreas, AreaNameArray, RetCode]
    if isinstance(ret, (list, tuple)):
        rc = int(ret[-1])
        names = None
        if len(ret) >= 2 and ret[1] is not None:
            try:
                names = list(ret[1])
            except Exception:
                names = ret[1]
        return (rc == 0), names, ret
    else:
        try:
            rc = int(ret)
            return (rc == 0), None, ret
        except Exception:
            return False, None, ret

In [49]:
# luego dividir (n1/n2 según tu criterio; antes usabas 4*n_pernos y 10)
ok_div, new_areas, raw = divide_area_by_edge_points(SapModel, "COL_FLANGE_TOP", n1=4*n_pernos, n2=10)
if not ok_div:
    print("Divide falló:", raw)
else:
    print("Divide OK, nuevas áreas:", new_areas)

Divide OK, nuevas áreas: ['643', '644', '645', '646', '647', '648', '649', '650']


In [50]:
z_col=10
SapModel.EditArea.Divide("643", 1, 0, [], 1, z_col)

[10, ('651', '652', '653', '654', '655', '656', '657', '658', '659', '660'), 0]